```{index} single: application; energy systems
```
```{index} single: solver; cbc
```
```{index} pandas dataframe
```
```{index} network optimization
```
```{index} stochastic optimization
```
```{index} SAA
```
```{index} linear decision rules
```

# Two-stage energy dispatch optimization

This notebook...

In [1]:
# install Pyomo and solvers
import requests
import types

url = "https://raw.githubusercontent.com/mobook/MO-book/main/python/helper.py"
helper = types.ModuleType("helper")
exec(requests.get(url).content, helper.__dict__)

helper.install_pyomo()
helper.install_cbc()

pyomo was previously installed
cbc was previously installed


True

# Preliminaries

The equilibrium point for the European power network, which operates on alternating current, is at a frequency of 50 Hertz with a tolerance threshold of +- 0.05 Hertz. Power network operators are responsible for maintaining a constant frequency on the network. The way that operators do this is mainly by perfectly balancing power supply and demand.

But what happens if power supply does not meet demand? When _supply exceeds demand_, then the electrical frequency increases. If _demand exceeds supply_, then the frequency drops. Since power plants are designed to operate within a certain frequency range, there is a risk that they will disconnect from the grid after a period of time. If the frequency deviates too much from the 50 Hertz, then there is a risk that the power plants switch off one after another,potentially leading to a complete power blackout.

Using conventional power generators such as coal and gas, power supply and demand are often matched by increasing/decreasing the production rate of their power plants and taking generating units on or off line. With the advent of renewable energy sources, it has become increasingly more difficult to match supply and demand. Besides not being controllable power sources, network operators rely on forecasting models to predict the power generated by renewable sources. In practice, this prediction is fairly accurate for solar energy, but wind energy is particularly difficult to predict correctly. 

<img src="https://www.mdpi.com/energies/energies-13-05595/article_deploy/html/images/energies-13-05595-g018.png" width="600"/>

The goal of this notebook is to ensure that power demand meets supply while taking into account wind fluctuations. We will introduce two optimization problems and solve them as stochastic optimization problems. **Read first the [energy dispatch problem](../04/power-network.ipynb) from the Chapter 4 for the preliminaries on power networks and the Optimal Power Flow (OPF) problem**. An important difference from the setting there, is that here we *will not assume that the wind generation is a decision variable*. Instead, wind generation is a random variable, as will be explained later on. We do assume that solar and hydro power are decision variables, since the former is accurately predicated whereas the latter is controllable.

<a name="uc"></a>
## Unit Commitment

We now consider another optimization problem relevant for energy systems named the _Unit Commitment (UC)_ problem. UC is an extended version of the OPF problem in which an additional binary decision variable $x_i$ is introduced to decide whether generator $i$ should be activated or not. In this formulation we include a fixed cost $\kappa_i$ that is incurred for the activation of a generator $i$, which is also added as a new parameter `c_fixed` to the network instance. 

In practice, the UC problem is often considered as a two-stage problem. Since it takes time to activate gas and coal generators before they can produce energy, this decision must be made in advance, i.e., as here-and-now decision in the first stage. In the second stage, we then decide the power generation levels of the (activated) coal and gas generators. Note that, in particular, *we cannot produce from generators we did not already turn on!* Lastly, the UC still includes the same physical constraints as the OPF, i.e., power generation limits and line capacity constraints.

All solar and hydro generators are activated by default. Solar power is treated as deterministic in the sense that in the instance that you are given it holds that $p_{\min}=p_{\max}$ for solar generators. Hydro generators are stilll controllable within their nontrivial generation limits.

The uncertainty will be described in terms of the _wind speed_ $v$ instead of the wind power. To determine the wind power resulting from a given wind speed, you need to use a so-called _power curve_ $g_i(\cdot)$ for a wind generator $i$, which is an explicit function that maps the wind speed $v$ to a wind power $g_i(v)$. In the network instance that you are given, there are two wind generators, one in node 64 which is an on-shore wind park, and one in node 65 which is a off-shore wind park. Being structuraly different, they have different power curves. See below a plot of the power curve function for the on-shore wind generator. 


<center><img src="onshore.png" width="700"/></center>


An analytical description of the power curves is given as follows

$$
\begin{align}
g_{64}(v) = 
\begin{cases}
0 & \text{if } v \leq 3 \\ 
0.16563 \cdot v^3 - 4.4718 & \text{if } 3 \leq v \leq 14 \\
450 & \text{if } v \geq 14
\end{cases}
\end{align}
$$

$$
\begin{align}
g_{65}(v) = 
\begin{cases}
0 & \text{if } v \leq 3.5 \\ 
0.18007 \cdot v^3 - 7.72049 & \text{if } 3.5 \leq v \leq 15 \\
600 & \text{if } v \geq 15
\end{cases}
\end{align}
$$

We now present the two-stage UC problem formulation. In the first stage, we decide which coal and gas generators to activate (all wind, solar and hydro generators are active by default). In the second stage, the exact power output of the wind parks is calculated using the power curves knowing the realization of the two wind speeds and the power outputs of the already activated generators needs to be adjusted to match demand exactly.

$$
\begin{align}
\begin{array}{llll}
\min        & \sum_{i \in \mathcal{G}^{\text{coal}} \cup \mathcal{G}^{\text{gas}}} \kappa_i x_i + \mathbb{E}_v Q(x, v) \\
\text{s.t.} & x_i \in \{0,1\} & \forall i \in \mathcal{G}^{\text{coal}} \cup \mathcal{G}^{\text{gas}}, 
\end{array}
\end{align}
$$

where

$$
\begin{align}
\begin{array}{lllll}
Q(x,v) := &\min        & \sum_{i \in \mathcal{G}^{\text{coal}} \cup \mathcal{G}^{\text{gas}}} c_i(p_i) \\
&\text{s.t.}
& p_i = g_i(v_i) & \forall i \in \mathcal{G}^{\text{wind}}\\
&& x_i p_{i}^{\min } \leq p_{i} \leq x_i p_{i}^{\max } & \forall i \in \mathcal{G}^{\text{coal}} \cup \mathcal{G}^{\text{gas}} \\
&& p_{i}^{\min } \leq p_{i} \leq p_{i}^{\max } & \forall i \in V \setminus (\mathcal{G}^{\text{coal}} \cup \mathcal{G}^{\text{gas}} \cup \mathcal{G}^{\text{wind}}) \\
&& \sum_{j: (i, j) \in E} f_{ij} - \sum_{j: (j, i) \in E} f_{ji} = p_i - d_i & \forall \, i \in V\\
&& f_{ij} =  b_{ij}(\theta_i - \theta_j), & \forall \, (i, j) \in E \\
&& -f_{ij}^{max} \leq f_{ij} \leq  f_{ij}^{\max}    & \forall (i, j) \in E\\
&& \theta_i \in \mathbb{R} & \forall i \in V \\
&& f_{ij} \in \mathbb{R}                 & \forall (i, j) \in E\\
&& p_{i} \geq 0                & \forall i \in V
\end{array}
\end{align}
$$

## Package and data import

In [3]:
# Load packages
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval as make_tuple
import networkx as nx
import time 

# Load solver
solver = pyo.SolverFactory('cbc')

# Download the data
nodes_df = pd.read_csv('nodes.csv', index_col=0)
edges_df = pd.read_csv('edges.csv', index_col=0)

# Read data
nodes = nodes_df.set_index("node_id").T.to_dict()
edges = edges_df.set_index(edges_df["edge_id"].apply(make_tuple)).T.to_dict()
network = {"nodes": nodes, "edges": edges}

### Network data

In this notebook, we consider only a single network instance, which be accessed in the `network` variable. Its data structure is identical to the that in [energy dispatch problem](../04/power-network.ipynb) and only the `p_max` and `f_max` values have been slightly modified to align the instance with the new stochastic problem setting. Moreover, we have added a new parameter `c_fixed` to the nodes data to account for activation costs of some of the generators.

We have left the wind generator data as part of the instance for completeness, so that one can access the wind nodes using the `energy_type` fields. Nodes `64` and `65` correspond to the two wind generators that this notebook focuses on.

In [4]:
nodes_df[nodes_df.is_generator]

,node_id,d,p_min,p_max,c_var,is_generator,energy_type,c_fixed
9,9,0.0,0.000000,400.000000,0.0,True,hydro,0.0
11,11,0.0,0.000000,200.000000,0.0,True,hydro,0.0
24,24,0.0,0.000000,422.086431,28.0,True,coal,1689.0
25,25,0.0,0.000000,227.384370,18.0,True,coal,1057.0
30,30,0.0,0.000000,235.306239,19.0,True,coal,1837.0
45,45,0.0,0.000000,371.349675,19.0,True,coal,1456.0
48,48,0.0,227.262510,227.262510,0.0,True,solar,0.0
53,53,0.0,97.526012,97.526012,0.0,True,solar,0.0
58,58,0.0,284.753966,284.753966,0.0,True,solar,0.0
60,60,0.0,98.693808,98.693808,0.0,True,solar,0.0


In [5]:
edges_df

,edge_id,b,f_max
0,"(0, 1)",10.0100,270.705509
1,"(0, 2)",23.5849,415.734756
2,"(3, 4)",125.3133,265.273978
3,"(2, 4)",9.2593,400.159230
4,"(4, 5)",18.5185,217.852748
...,...,...,...
174,"(64, 65)",28.9059,1200.000000
175,"(67, 68)",28.9059,1200.000000
176,"(80, 79)",28.9059,1200.000000
177,"(86, 85)",4.8216,602.814908


Wind turbines risk to break if the wind speed is too high, so wind turbines need to be _curtailed_ (i.e., deactivated) once the wind speed exceeds a specified maximum speed $v_{\max}$. An additional second stage decision is therefore needed to decide whether to curtail the wind turbines or not. Add this additional requirement to the presented second-stage formulation.

  - Write out the extensive form assuming that you have $T$ samples. Explain what you did.
  - Assume that 
    - $v_{64} \sim \text{Weibull}$ with scale parameter 15 and shape parameter 2.6.
    - $v_{65} \sim \text{Weibull}$ with scale parameter 18 and shape parameter 3.0.  

    Sample 100 data points and implement the extensive form problem in pyomo and report the objective value.
  - Using the previous 100 data points, compute the average wind speeds. Then compute the objective value.
  - Analyze the differences in the objective value and explain the differences.

In [ ]:
def g_offshore(v):
    if v <= 3.5:
      return 0
    elif v > 15:
      return 600
    else:
      return 0.18007 * v**3 - 7.72049

def g_onshore(v):
    if v <= 3:
      return 0
    elif v > 14:
      return 450
    else:
      return 0.16563 * v**3 - 4.4718

g = {64: g_onshore, 65: g_offshore}

In [ ]:
def Q2(network, samples):
    """
    Input:
    - network: a dictionary containing:
      - a dictionary of nodes with a dictionary of attributes
      - a dictionary of edges with a dictionary of attributes

    Output:
    - power_generation: a dictionary containing the power generation for each node
    - power_flows: a dictionary containing the power flow for each edge
    """
    # Given functions
    vmax = {64: 20, 65: 22}
    # g = lambda x: x

    # Define a model
    model = pyo.ConcreteModel("Q1")
    
    # Define sets
    model.T = pyo.Set(initialize=range(len(samples)))
    model.V = pyo.Set(initialize=network["nodes"].keys())
    model.E = pyo.Set(initialize=network["edges"].keys())
    model.W = pyo.Set(initialize=[i for i, data in network["nodes"].items() if data['energy_type'] == 'wind'])
    model.NW = pyo.Set(initialize=[i for i, data in network["nodes"].items() if data['energy_type'] != 'wind']) # Not wind
    model.M = 1000

    # Declare decision variables
    model.x = pyo.Var(V, domain=pyo.Binary)
    model.p = pyo.Var(V, T, domain=pyo.NonNegativeReals)
    model.theta = pyo.Var(V, T, domain=pyo.Reals)
    model.f = pyo.Var(E, T, domain=pyo.Reals)
    model.y = pyo.Var(W, T, domain=pyo.Binary)

    # Declare objective value data["c_fixed"]
    model.first_stage_objective = pyo.Expre
    model.objective = pyo.Objective(expr = sum(1000 * model.x[i] for i, data in network["nodes"].items() if data['energy_type'] in ['coal', 'gas']) +
                                    1/len(T) * sum(sum(data["c_var"] * model.p[i, t] for i, data in network["nodes"].items() if data["is_generator"]) for t in T),
                                    sense=pyo.minimize)

    # Declare constraints
    model.wind_speed_to_power = pyo.Constraint(W, T, rule=lambda m, i, t: model.p[i, t] == (1 - model.y[i, t]) * g[i](samples[t][i]))
    model.wind_curtailment = pyo.Constraint(W, T, rule=lambda m, i, t: samples[t][i] <= vmax[i] + model.y[i, t] * M)

    model.generation_upper_bound = pyo.Constraint(NW, T, rule=lambda m, i, t: m.p[i, t] <= m.x[i] * network["nodes"][i]["p_max"])
    model.generation_lower_bound = pyo.Constraint(NW, T, rule=lambda m, i, t: m.x[i] * network["nodes"][i]["p_min"] <= m.p[i, t])

    model.outgoing_flow = pyo.Expression(V, T, rule=lambda m, i, t: sum(m.f[i, j, t] for j in V if (i, j) in E))
    model.incoming_flow = pyo.Expression(V, T, rule=lambda m, i, t: sum(m.f[j, i, t] for j in V if (j, i) in E))
    model.flow_conservation = pyo.Constraint(V, T, rule=lambda m, i, t: m.incoming_flow[i, t] - m.outgoing_flow[i, t] == m.p[i, t] - nodes[i]["d"])
    model.susceptance = pyo.Constraint(E, T, rule=lambda m, i, j, t: m.f[(i, j), t] == network["edges"][(i, j)]["b"] * (m.theta[i, t] - m.theta[j, t]))

    model.flows_upper_bound = pyo.Constraint(E, T, rule=lambda m, i, j, t: m.f[(i, j), t] <= network["edges"][(i, j)]["f_max"])
    model.flows_lower_bound = pyo.Constraint(E, T, rule=lambda m, i, j, t: - m.f[(i, j), t] <= network["edges"][(i, j)]["f_max"])


    # Solve
    result = solver.solve(model)

    # Print solution
    print(f"Solver status: {result.solver.status}, {result.solver.termination_condition}")
    print(np.sum([model.y[x,t].value for x,t in model.y]))

    return model

We sample from two different continuous Weibull distributions with different scale and shape parameters and then solve the problem.

In [ ]:
seed = 1
rng = np.random.default_rng(seed)

scale64 = 15
shape64 = 2.6
scale65 = 8
shape65 = 3

samples = [{64:  scale64*rng.weibull(shape64), 65: scale65*rng.weibull(shape65)} for i in range(100)]
model = Q2(network, samples)
print(f"Objective value: {model.objective()}")
# [(i, model.x[i].value) for i in model.x]